## TensorFlow and Model Loading Configuration

In [1]:
import os
import tensorflow as tf
# Load compressed models from tensorflow_hub
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'

## Visualization and Utility Imports

In [ ]:
import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12, 12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import PIL.Image
import time
import functools

## Tensor to Image Conversion

In [ ]:
def tensor_to_image(tensor):
    """
    Converts a TensorFlow tensor to a PIL image.

    Args:
        tensor (tf.Tensor): Input tensor.

    Returns:
        PIL.Image.Image: Converted PIL image.
    """
    tensor = tensor * 255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor) > 3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)


## Image Loading Function

In [2]:
def nst_load_img(path_to_img):
    """
    Loads an image from the given file path and performs preprocessing.

    Args:
        path_to_img (str): File path of the input image.

    Returns:
        tf.Tensor: Preprocessed image tensor.
    """
    max_dim = 512

    # Read image file
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32)

    # Resize image
    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]

    return img


## Image Visualization Function

In [ ]:
def show_image(image, title=None):
    """
    Displays an image using matplotlib.pyplot.show_image.

    Args:
        image (tf.Tensor or np.ndarray): Image to display.
        title (str, optional): Title of the image plot.

    Returns:
        The function shows the image
    """
    if len(image.shape) > 3:
        image = tf.squeeze(image, axis=0)

    plt.imshow(image)
    if title:
        plt.title(title)


In [4]:
content_image = nst_load_img('/home/ritesh/Desktop/workENV/atithi_work/Semantic_Segmentation/dataset_combined_final/images/image_1.png')
style_image = nst_load_img('/home/ritesh/Desktop/workENV/atithi_work/style_transfer/class_based_style_transfer/floori_styles/style_4.png')

#plt.subplot(1, 2, 1)
#show_image(content_image, 'Content Image')

#plt.subplot(1, 2, 2)
#show_image(style_image, 'Style Image')

## TensorFlow Hub and Arbitrary Image Stylization Model

In [5]:
import tensorflow_hub as hub
import matplotlib.pyplot as plt

# Load the arbitrary image stylization model from TensorFlow Hub
hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

# Apply image stylization by passing the content and style images to the model
stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]

# Convert the stylized image tensor to tf.uint8 data type for saving
stylized_image = tf.image.convert_image_dtype(stylized_image, tf.uint8)

# Remove the singleton dimensions and obtain a 3-dimensional image tensor
stylized_image = tf.squeeze(stylized_image)

# Convert the stylized image tensor to a NumPy array
stylized_image = stylized_image.numpy()

# Save the stylized image as a PNG file
output_path = 'try_1.png'
plt.imsave(output_path, stylized_image)
